# Choosing a Location for Drilling

**Objective:** 

You are provided with oil samples from three regions, each containing 10,000 deposits where the quality of the oil and the volume of reserves have been measured. Build a machine learning model to determine which region will yield the highest profit from extraction. Analyze the potential profit and risks using the *Bootstrap* technique.

**Steps to Determine the Optimal Location:**

- In the selected region, identify the deposits and determine the feature values for each.
- Build a model and estimate the volume of reserves.
- Select deposits with the highest estimated values. The number of deposits will depend on the company's budget and the cost of drilling each well.
- Profit equals the total profit from the selected deposits.

## Loading and Preparing the Data

In [1]:
# Importing All Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Let's Take a Look at the Datasets
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
print(data_0.head())

data_1 = pd.read_csv('/datasets/geo_data_1.csv')
print(data_1.head())

data_2 = pd.read_csv('/datasets/geo_data_2.csv')
print(data_2.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [3]:
# Exploring the Datasets
data_0.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99990,NaN,NaN,NaN,NaN
top,HZww2,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.500419,0.250143,2.502647,92.500000
std,NaN,0.871832,0.504433,3.248248,44.288691
min,NaN,-1.408605,-0.848218,-12.088328,0.000000
25%,NaN,-0.072580,-0.200881,0.287748,56.497507
50%,NaN,0.502360,0.250252,2.515969,91.849972
75%,NaN,1.073581,0.700646,4.715088,128.564089


In [4]:
data_1.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,wt4Uk,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,1.141296,-4.796579,2.494541,68.825000
std,NaN,8.965932,5.119872,1.703572,45.944423
min,NaN,-31.609576,-26.358598,-0.018144,0.000000
25%,NaN,-6.298551,-8.267985,1.000021,26.953261
50%,NaN,1.153055,-4.813172,2.011479,57.085625
75%,NaN,8.621015,-1.332816,3.999904,107.813044


In [5]:
data_2.describe(include='all')

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,KUPhW,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.002023,-0.002081,2.495128,95.000000
std,NaN,1.732045,1.730417,3.473445,44.749921
min,NaN,-8.760004,-7.084020,-11.970335,0.000000
25%,NaN,-1.162288,-1.174820,0.130359,59.450441
50%,NaN,0.009424,-0.009482,2.484236,94.925613
75%,NaN,1.158535,1.163678,4.858794,130.595027


I noticed an anomaly in the dataset: the number of elements in the `id` column and the number of unique values in it do not match in each dataset. This could be due to several reasons: there might be missing values in the dataset, duplicate rows, or data points could have been split across multiple rows.

We need to investigate the cause.

In [6]:
# Searching for Duplicates
print(data_0[data_0.duplicated()])
print(data_1[data_1.duplicated()])
print(data_2[data_2.duplicated()])

Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


No duplicates were found.

In [7]:
print('data_0')
for i in ['id', 'f0', 'f1', 'f2', 'product']:
    print(i, ':', data_0[i].isna().sum())

print('data_1')
for i in ['id', 'f0', 'f1', 'f2', 'product']:
    print(i, ':', data_1[i].isna().sum())

print('data_2')
for i in ['id', 'f0', 'f1', 'f2', 'product']:
    print(i, ':', data_2[i].isna().sum())

data_0
id : 0
f0 : 0
f1 : 0
f2 : 0
product : 0
data_1
id : 0
f0 : 0
f1 : 0
f2 : 0
product : 0
data_2
id : 0
f0 : 0
f1 : 0
f2 : 0
product : 0


There are no missing values. The remaining issue is that data for a single point may have been split across multiple rows. We will correct this.

In [8]:
# Grouping and Summarizing the Data
data_0 = data_0.groupby('id').sum().reset_index()
data_1 = data_1.groupby('id').sum().reset_index()
data_2 = data_2.groupby('id').sum().reset_index()

**Conclusion:**

The data check was successful. All datasets were loaded and reviewed. An anomaly in the `id` column was detected and corrected.

## Training and Evaluating the Model

In [9]:
# Extracting Features and Targets from All Datasets
target_0 = data_0['product']
target_1 = data_1['product']
target_2 = data_2['product']
features_0 = data_0.drop(['product', 'id'], axis=1)
features_1 = data_1.drop(['product', 'id'], axis=1)
features_2 = data_2.drop(['product', 'id'], axis=1)

In [10]:
features_train_0, features_valid_0, target_train_0, target_valid_0 =\
train_test_split(features_0, target_0, test_size=0.25, random_state=404540)

features_train_1, features_valid_1, target_train_1, target_valid_1 =\
train_test_split(features_1, target_1, test_size=0.25, random_state=404540)

features_train_2, features_valid_2, target_train_2, target_valid_2 =\
train_test_split(features_2, target_2, test_size=0.25, random_state=404540)

In [11]:
# Let's Check
print(features_train_0.count())
features_valid_0.count()

f0    74992
f1    74992
f2    74992
dtype: int64


f0    24998
f1    24998
f2    24998
dtype: int64

In [12]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
prediction_0 = model.predict(features_valid_0)
error = mean_squared_error(target_valid_0, prediction_0) ** 0.5
print('реальное среднее региона:', target_valid_0.mean())
print('rmse:', error)
print('среднее региона:', prediction_0.mean())

реальное среднее региона: 92.37590379111649
rmse: 37.91941705115252
среднее региона: 92.69152797933344


In [13]:
model.fit(features_train_1, target_train_1)
prediction_1 = model.predict(features_valid_1)
error = mean_squared_error(target_valid_1, prediction_1) ** 0.5
print('реальное среднее региона:', target_valid_1.mean())
print('rmse:', error)
print('среднее региона:', prediction_1.mean())

реальное среднее региона: 69.00661381144667
rmse: 0.8937225224888704
среднее региона: 68.99871110680868


In [14]:
model.fit(features_train_2, target_train_2)
prediction_2 = model.predict(features_valid_2)
error = mean_squared_error(target_valid_2, prediction_2) ** 0.5
print('реальное среднее региона:', target_valid_2.mean())
print('rmse', error)
print('среднее региона:', prediction_2.mean())

реальное среднее региона: 95.28390554153665
rmse 39.957916008212315
среднее региона: 95.0941215156349


**Conclusion:**

The data was split into validation and training sets in a 25:75 ratio. The models were trained and results were obtained.

The RMSE of the models for the first and third regions is twice as low as the real average region, indicating high prediction accuracy. However, the best predictions were achieved with the model trained on the data from the second region, with an RMSE of 0.89, reflecting very high accuracy.

Regarding the average predicted reserves, the first and third regions showed similar results of approximately 90,000 barrels, while the second region averaged 68,000 barrels per point. From these results, it can be inferred that the first and third regions are more likely to be profitable. However, further investigation is required for more accurate conclusions.

## Preparing for Profit Calculation

In [15]:
n = 200
budget= 10e9
bar_cost = 450000

In [16]:
budget / (bar_cost * 200)

111.11111111111111

**Conclusion**

For a region to be profitable, it should have an average of 112,000 barrels per point. None of the regions met this criterion. Further investigation is needed.

## Calculating Profit and Risks

In [17]:
def indexes(prediction, target):
    target = target.reset_index(drop=True)
    prediction = pd.Series(prediction)
    return prediction, target


prediction_0, target_valid_0 = indexes(prediction_0, target_valid_0)
prediction_1, target_valid_1 = indexes(prediction_1, target_valid_1)
prediction_2, target_valid_2 = indexes(prediction_2, target_valid_2)

In [18]:
def profit(target, prediction):
    pred_sorted = prediction.sort_values(ascending=False)
    target_sorted = target[pred_sorted.index][:200]
    product = target_sorted.sum()
    return product * bar_cost - budget

In [21]:
profit(target_valid_0, prediction_0) 

3040078284.236227

In [22]:
profit(target_valid_1, prediction_1) 

2427215664.1457024

In [23]:
profit(target_valid_2, prediction_2) 

2634774256.896986

**Conclusion**

The first and second regions were found to be profitable, while surprisingly, the third region appears to be entirely unprofitable. Even the richest points in the third region will not cover the budget. Further investigation will be conducted.

In [24]:
state = np.random.RandomState(12345)
def interval_risk(target, prediction):
    list_profit = []
    count = 0
    for i in range(1000):
        target_500 = target.sample(500, replace=True, random_state=state)
        prediction_500 = prediction[target_500.index]
        prof = profit(target_500, prediction_500)
        list_profit.append(prof)
        if prof < 0:
            count += 1
    list_profit =  pd.Series(list_profit)
    pvalue = 1. * count / 1000
    lower = list_profit.quantile(0.025)
    upper = list_profit.quantile(0.975)
    
    return lower, upper, pvalue, list_profit.mean()

In [25]:
lower, upper, pvalue, mean = interval_risk(target_valid_0, prediction_0)
print('95% доверительный интервал от', lower, 'до', upper)
print('шанс риска:', pvalue)
print('средняя прибыль:', mean)

95% доверительный интервал от -57941948.072227135 до 938368139.894533
шанс риска: 0.046
средняя прибыль: 423371145.51662904


In [26]:
lower, upper, pvalue, mean = interval_risk(target_valid_1, prediction_1)
print('95% доверительный интервал от', lower, 'до', upper)
print('шанс риска:', pvalue)
print('средняя прибыль:', mean)

95% доверительный интервал от 83827620.25552034 до 929397032.025899
шанс риска: 0.01
средняя прибыль: 502512797.33764553


In [27]:
# With the third region, it is clear that it is unprofitable in any case,
# but let's still test the function on it to ensure its correctness
lower, upper, pvalue, mean = interval_risk(target_valid_2, prediction_2)
print('95% доверительный интервал от', lower, 'до', upper)
print('шанс риска:', pvalue)
print('средняя прибыль:', mean)

95% доверительный интервал от -114675507.1495636 до 973753888.0273303
шанс риска: 0.064
средняя прибыль: 426717005.04442984


**Conclusion**

The second region has a confidence interval exclusively with positive profit values, which is a good sign. In contrast, the first and third regions have intervals with negative values. The probability of loss is 5% for the first region, 0.7% for the second region, and 100% for the third region. The second region meets the client's criteria, with a risk of loss of no more than 2.5%.

My recommendation is to start exploring the second region. A more detailed rationale will be provided in the final summary.

## General Conclusion:

**Summary of Work:**

- Opened the file, checked its integrity, and corrected anomalies in the datasets.
- Split the data into validation and training datasets with a 25:75 ratio.
- Trained the model, predicted the target feature, analyzed the model's performance, and concluded that the model was of high quality.
- Developed several functions for calculating profit, confidence intervals, and the likelihood of loss for each region.
- Applied these functions to resolve the issue.

**Conclusion:**

By analyzing the average oil reserves in different regions, I initially concluded that the first and third regions could be promising for the client due to their higher average reserves. Conversely, the second region had the lowest average reserves, which might not guarantee profitability.

However, after conducting further analysis, the results changed. It was found that only the second region is likely to be profitable, while the first and third regions turned out to be unprofitable. Even with the 200 largest oil fields in the third region, it will not be able to fully cover its expenses.

I strongly recommend focusing on the second region. Firstly, the risk of losses in the second region is significantly lower than in the others. Secondly, the 95% confidence interval for the second region includes only positive profit values, which substantially reduces the risk of loss.